<a href="https://colab.research.google.com/github/AdlaiSantopadre/Home/blob/main/Project/TrackManutenzioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questo blocco note sviluppa un foglio di calcolo excel per il tracciamento delle attività di manutenzione programmata prevista per le dotazioni antincendio in ambito autostradale in cui  sono presenti siti di servizio, locali tecnici, gallerie (ed altri ambienti).I dispositivi previsiti si trovano dislocati in itinere lungo le carreggiate sinistra e destrae in corrispondenza di altre pertinenze come le stazioni di pedaggio e le aree di servizio. Questi prevedono la presenza di una serie di dispositivi le cui tipologie si possono ricondurre all'elenco sottostante:

E' prevista  la presenza di una serie di dispositivi le cui tipologie si possono ricondurre all'elenco sottostante di **Tipo Risorsa**:


1 Portoni carrabile REI

2 Porte REI

3 Attacchi per autopompa

4 Idranti

5 Cassette antincendio

6 Gruppi pressurizzazione antincendio

7 Impianti rilevamento incendio

8 Serrande REI

9 Estintori

10 Supporti estintore

Ogni dispositivo è identificato univocamente da un **Identificativo numerico** , si trova in un luogo georeferenziato tramite un etichetta (tag) ed è a sua volta associato ad un etichetta georeferenziata. L'elenco delle tipologie dei luoghi si può ricondurre alla lista sottostante:

Ogni dispositivo localizzato mediante etichetta georeferenziata può essere individuato mediante un campo che descrive la localizzazione.
L'elenco delle tipologie dei luoghi (**Tipo luogo**) si può ricondurre alla lista sottostante:

1 Area di servizio

2 Fabbricato %% accessorio

3 Locale Tecnico %% accessorio

4 Radio Ripetitore di Altura

5 Shelter Impianti

6 Sito operativo (Hub)

7 Stazione di esazione

8 Galleria

Seguono i dati per la georeferenziazione

Per ogni tipologia di dispositivo antincendio (= Tipo Risorsa) sussistono attività di manutenzione (ordinaria) programmata.
Sostanzialmente una attività di manutenzione si contraddistingue (oltre che **tipo di attività** da svolgere) per:
dispositivo associato;
periodicità;
data di inizio attività (timestamp che la identifica univocamente) ;
data di scadenza attività;
data di effettuazione.
Sulla base della data di effettuazione e della periodicità viene calcolato il timestamp della successiva manutenzione periodica.
Ogni attività prevede un assegnatario, prevede una flag di stato in relazione all'assegnamento ad una risorsa di manodopera (da assegnare/assegnata) ed una flag di stato rispetto all'esecuzione(da eseguire/eseguita)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NON ESEGUIRE PIU`
Il codice Python seguente è una analisi raffinata dei dataframe precedenti:
genera un foglio di calcolo con 5 schede: Una per i *dispositiv*i, una per (il tipo di ) *manutenzioni* ordinarie , una per la *periodicità* di esse , una per i *luoghi*, una per (il tipo di)*risorse*  e infine una per (il tracking delle ) *attività* di manutenzione.


In [ ]:
"""
import pandas as pd
from datetime import datetime, timedelta
#La libreria datetime viene importata per gestire univocità e cronologia delle attività
# Definire alcune date di esempio per le scadenze periodiche
today = datetime.today()
dates = {
    "Trimestrale": today + timedelta(days=90),
    "Semestrale": today + timedelta(days=180),
    "Annuale": today + timedelta(days=365)
}

# Creazione dei DataFrame iniziali
dispositivi_data = {
    "ID Dispositivo": ["D001", "D002", "D003", "D004"],
    "Etichetta": ["Tag1", "Tag2", "Tag3", "Tag4"],
    "Tipo Risorsa": [None, None, None, None],  # da lista
    "Tipo Luogo": [None, None, None, None],  # da lista
    "Etichetta Georeferenziata": ["Geo1", "Geo2", "Geo3", "Geo4"],
}
dispositivi_df = pd.DataFrame(dispositivi_data)
#
manutenzioni_data = {
    "Tipo Attività": ["A001", "A002", "A003", "A004"],  # da lista
    "Periodicità Manutenzione": [None, None, None, None],  # Campo associato da periodicità
}
manutenzioni_df = pd.DataFrame(manutenzioni_data)
#
periodicità_data = {
    "Periodicità Manutenzione": [90, 180, 365],
}
periodicità_df = pd.DataFrame(periodicità_data)
#
luoghi_data = {
    "Tipo Luogo": ["Area di servizio", "Fabbricato", "Locale Tecnico", "Radio Ripetitore di Altura", "Shelter Impianti", "Sito operativo (Hub)", "Stazione di esazione", "Galleria"],
}
luoghi_df = pd.DataFrame(luoghi_data)
#
risorse_data = {
    "Tipo Risorsa": ["Portoni carrabili REI", "Porte REI", "Attacchi per autopompa", "Idranti", "Cassette antincendio", "Gruppi pressurizzazione antincendio", "Impianti rilevamento incendio", "Serrande REI", "Estintori", "Supporti estintori"],
}
risorse_df = pd.DataFrame(risorse_data)
#
attività_data = {
    "Codice Attività": [today.timestamp(), today.timestamp() + 1, today.timestamp() + 2, today.timestamp() + 3],
    "ID Dispositivo": ["D001", "D002", "D003", "D004"],  # Campo associato dal DataFrame Dispositivi
    "Periodicità": [None, None, None, None],  # campo associato da periodicità
    "Inizio Manutenzione": ["2023-01-15", "2023-02-20", "2023-03-25", "2023-04-30"],
    "Data Scadenza": [dates["Trimestrale"], dates["Semestrale"], dates["Annuale"], dates["Trimestrale"]],
    "Flag scadenza": [None, None, None, None],
    "Flag assegnazione": [None, None, None, None],
}
attività_df = pd.DataFrame(attività_data)
#
# Funzione per aggiungere dispositivi con controllo su Tipo Luogo e Tipo Risorsa
def Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, id_dispositivo, etichetta, tipo_luogo, tipo_risorsa, etichetta_georeferenziata):
    if tipo_luogo not in luoghi_df["Tipo Luogo"].values:
        raise ValueError(f"Tipo Luogo '{tipo_luogo}' non valido. Deve essere uno tra {luoghi_df['Tipo Luogo'].values}")
    if tipo_risorsa not in risorse_df["Tipo Risorsa"].values:
        raise ValueError(f"Tipo Risorsa '{tipo_risorsa}' non valido. Deve essere uno tra {risorse_df['Tipo Risorsa'].values}")

    new_data = {
        "ID Dispositivo": id_dispositivo,
        "Etichetta": etichetta,
        "Tipo Risorsa": tipo_risorsa,
        "Tipo Luogo": tipo_luogo,
        "Etichetta Georeferenziata": etichetta_georeferenziata,
    }

    # Use pd.concat instead of the deprecated .append method
    return pd.concat([dispositivi_df, pd.DataFrame([new_data])], ignore_index=True)

# Esempio di utilizzo della funzione Aggiungi_dispositivo
dispositivi_df = Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, "D005", "Tag5", "Fabbricato", "Estintori", "Geo5")

# Visualizzazione del DataFrame aggiornato
print(dispositivi_df)



#Scrittura del file Excel con multi-schede
with pd.ExcelWriter('/content/drive/My Drive/Dispositivi_Antincendio_Manutenzione.xlsx') as writer:
  dispositivi_df.to_excel(writer, sheet_name='Dispositivi', index=False)
  attività_df.to_excel(writer, sheet_name='Attività di Manutenzione', index=False)
  manutenzioni_df.to_excel(writer, sheet_name='Manutenzioni ordinarie', index=False)
  periodicità_df.to_excel(writer, sheet_name='Periodicità', index=False)
  luoghi_df.to_excel(writer, sheet_name='Luoghi', index=False)
  risorse_df.to_excel(writer, sheet_name='Risorse', index=False)
"""



A questo punto passo a una versione con DataFrame inizialmente vuoti

In [ ]:
import panda as pd
#La libreria datetime viene importata per gestire univocità e cronologia delle attività
from datetime import datetime, timedelta

# Definire alcune date di esempio per le scadenze periodiche
today = datetime.today()
dates = {
    "Trimestrale": today + timedelta(days=90),
    "Semestrale": today + timedelta(days=180),
    "Annuale": today + timedelta(days=365)
}

# Creazione di DataFrame vuoti
dispositivi_df = pd.DataFrame(columns=["ID Dispositivo", "Etichetta", "Tipo Risorsa", "Tipo Luogo", "Etichetta Georeferenziata"])

manutenzioni_df = pd.DataFrame(columns=["Tipo Attività", "Periodicità Manutenzione"])

periodicità_df = pd.DataFrame(columns=["Periodicità Manutenzione"])

luoghi_df = pd.DataFrame(columns=["Tipo Luogo"])

risorse_df = pd.DataFrame(columns=["Tipo Risorsa"])

attività_df = pd.DataFrame(columns=["Codice Attività", "ID Dispositivo", "Periodicità", "Inizio Manutenzione", "Data Scadenza", "Flag scadenza", "Flag assegnazione"])

#Scrittura del file Excel con multi-schede
with pd.ExcelWriter('/content/drive/My Drive/Dispositivi_Antincendio_Manutenzione.xlsx') as writer:
  dispositivi_df.to_excel(writer, sheet_name='Dispositivi', index=False)
  attività_df.to_excel(writer, sheet_name='Attività di Manutenzione', index=False)
  manutenzioni_df.to_excel(writer, sheet_name='Manutenzioni ordinarie', index=False)
  periodicità_df.to_excel(writer, sheet_name='Periodicità', index=False)

Funzione **Aggiungi_dispositivo**:

Questa funzione prende come input il DataFrame dei dispositivi (dispositivi_df), i DataFrame contenenti i luoghi (luoghi_df) e le risorse (risorse_df), e i dettagli del nuovo dispositivo da aggiungere.
Controllo dei Valori: La funzione verifica che i valori di tipo_luogo e tipo_risorsa appartengano ai valori predefiniti nei rispettivi DataFrame (luoghi_df e risorse_df). Se un valore non è valido, viene sollevata un'eccezione (ValueError).
Aggiunta del Dispositivo: Se i controlli sono superati, il nuovo dispositivo viene aggiunto al DataFrame dispositivi_df.

Per utilizzare questa funzione, mi assicuro di popolare prima i DataFrame luoghi_df e risorse_df con i dati appropriati.

In [ ]:
#
luoghi_data = {
    "Tipo Luogo": ["Area di servizio", "Fabbricato", "Locale Tecnico", "Radio Ripetitore di Altura", "Shelter Impianti", "Sito operativo (Hub)", "Stazione di esazione", "Galleria"],
}
luoghi_df = pd.DataFrame(luoghi_data)
#
risorse_data = {
    "Tipo Risorsa": ["Portoni carrabili REI", "Porte REI", "Attacchi per autopompa", "Idranti", "Cassette antincendio", "Gruppi pressurizzazione antincendio", "Impianti rilevamento incendio", "Serrande REI", "Estintori", "Supporti estintori"],
}
risorse_df = pd.DataFrame(risorse_data)
#
print(luoghi_df)
print(risorse_df)
print(dispositivi_df)

                   Tipo Luogo
0            Area di servizio
1                  Fabbricato
2              Locale Tecnico
3  Radio Ripetitore di Altura
4            Shelter Impianti
5        Sito operativo (Hub)
6        Stazione di esazione
7                    Galleria
                          Tipo Risorsa
0                Portoni carrabili REI
1                            Porte REI
2               Attacchi per autopompa
3                              Idranti
4                 Cassette antincendio
5  Gruppi pressurizzazione antincendio
6        Impianti rilevamento incendio
7                         Serrande REI
8                            Estintori
9                   Supporti estintori
Empty DataFrame
Columns: [ID Dispositivo, Etichetta, Tipo Risorsa, Tipo Luogo, Etichetta Georeferenziata]
Index: []


In [ ]:

# Funzione per aggiungere dispositivi con controllo su Tipo Luogo e Tipo Risorsa
def Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, id_dispositivo, etichetta, tipo_luogo, tipo_risorsa, etichetta_georeferenziata):
    if tipo_luogo not in luoghi_df["Tipo Luogo"].values:
        raise ValueError(f"Tipo Luogo '{tipo_luogo}' non valido. Deve essere uno tra {luoghi_df['Tipo Luogo'].values}")
    if tipo_risorsa not in risorse_df["Tipo Risorsa"].values:
        raise ValueError(f"Tipo Risorsa '{tipo_risorsa}' non valido. Deve essere uno tra {risorse_df['Tipo Risorsa'].values}")

    new_data = {
        "ID Dispositivo": id_dispositivo,
        "Etichetta": etichetta,
        "Tipo Risorsa": tipo_risorsa,
        "Tipo Luogo": tipo_luogo,
        "Etichetta Georeferenziata": etichetta_georeferenziata,
    }
    return pd.concat([dispositivi_df, pd.DataFrame([new_data])], ignore_index=True)


# Esempio di utilizzo della funzione Aggiungi_dispositivo
#Per utilizzare questa funzione, assicurati di popolare prima i DataFrame luoghi_df e risorse_df con i dati appropriati.
dispositivi_df = Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, "D005", "Tag5", "Fabbricato", "Estintori", "Geo5")

# Visualizzazione del DataFrame aggiornato
print(dispositivi_df)

  ID Dispositivo Etichetta Tipo Risorsa  Tipo Luogo Etichetta Georeferenziata
0           D005      Tag5    Estintori  Fabbricato                      Geo5


**ESEGUIRE LA PRECEDENTE CELLA E POI PROSEGUIRE**

In [ ]:
"""
from google.colab import sheets

sheet = sheets.InteractiveSheet(df=dispositivi_df)
#sheet = sheets.InteractiveSheet(df=attività_df)
"""

'\nfrom google.colab import sheets\n\nsheet = sheets.InteractiveSheet(df=dispositivi_df)\n#sheet = sheets.InteractiveSheet(df=attività_df)\n'

Sostituire i valori utilizzati nel DataFrame dispositivi_data nel campo ID_dispositivo con un valore univoco utilizzando gli UUID.Quindi modifico la funzione Aggiungi_dispositivo

In [ ]:
import uuid
# Funzione per aggiungere dispositivi con controllo su Tipo Luogo e Tipo Risorsa
def Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, id_dispositivo, etichetta, tipo_luogo, tipo_risorsa, etichetta_georeferenziata):
    if tipo_luogo not in luoghi_df["Tipo Luogo"].values:
        raise ValueError(f"Tipo Luogo '{tipo_luogo}' non valido. Deve essere uno tra {luoghi_df['Tipo Luogo'].values}")
    if tipo_risorsa not in risorse_df["Tipo Risorsa"].values:
        raise ValueError(f"Tipo Risorsa '{tipo_risorsa}' non valido. Deve essere uno tra {risorse_df['Tipo Risorsa'].values}")
    new_id = uuid.uuid1()  # Generazione di un UUID univoco
    new_data = {
        "ID Dispositivo": new_id,
        "Etichetta": etichetta,
        "Tipo Risorsa": tipo_risorsa,
        "Tipo Luogo": tipo_luogo,
        "Etichetta Georeferenziata": etichetta_georeferenziata,
    }
    return pd.concat([dispositivi_df, pd.DataFrame([new_data])], ignore_index=True)


# Esempio di utilizzo della funzione Aggiungi_dispositivo
#Per utilizzare questa funzione, assicurati di popolare prima i DataFrame luoghi_df e risorse_df con i dati appropriati.
dispositivi_df = Aggiungi_dispositivo(dispositivi_df, luoghi_df, risorse_df, "D005", "Tag5", "Fabbricato", "Estintori", "Geo5")

# Visualizzazione del DataFrame aggiornato
print(dispositivi_df)

                         ID Dispositivo Etichetta Tipo Risorsa  Tipo Luogo  \
0                                  D005      Tag5    Estintori  Fabbricato   
1  029847dc-6847-11ef-94e1-0242ac1c000c      Tag5    Estintori  Fabbricato   

  Etichetta Georeferenziata  
0                      Geo5  
1                      Geo5  


Mentre l'aggiunta di un dispositivo mediante la funzione **Aggiungi_dispositivo**,permette ad ora di inserire un dispositivo  identificato univocamente, associato ad un tipo di luogo ed ad un tipo di risorsa, rimangono alcuni aspetti da implementare.
Una funzione **Georeferenzia_dispositivo** modificando il *Dataframe dispositivi* deve permettere di inserire l'etichetta univoca  per la georeferenziazione (può essere unizialmente un campo testuale che descrive la localizzazione della risorsa ed è sottoposto a test di unicità),
In ogni caso andra`risolta la costruzione di una mappa (ID_dispositivo -coordinate terrestri) per ottimizzazioni di svolgimento delle manutenzioni ordinarie

Il dataframe attività deve essere popolaato di tutti i record relativi ai dispositivi, con un record per ogni manutenzione prevista per esso attivata con *data di inizio* cui si associa un *identificatore univoco per la attività* dato dal suo timestamp, il dato della *periodicità* scelta dal *Dataframe periodicità *la scadenza calcolata in base alla periodicità e le due flag (*stato di assegnazione* e *stato di esecuzione*)che hanno come valori di default `non_assegnato`  e `non_eseguito`.
Particolare importanza assume la funzione **Aggiorna_attività** qualora modifichi la *flag sullo stato di esecuzione*in `eseguita` che sulla base della periodicità presente genera una nuovo record con inizio attività pari all`attuale timestamp